In [ ]:
import requests

from bs4 import BeautifulSoup

from chatcrawler.logger import logger

from urllib.parse import urlparse
import threading
import os
import ssl
from chatcrawler.hyperlink import get_domain_hyperlinks
from chatcrawler.utils import read_pdf_from_url

ssl._create_default_https_context = ssl._create_unverified_context

# Define root domain to crawl
domain = "openai.com"
full_url = f"https://{domain}/"

import threading
from queue import Queue


def worker(queue: Queue, local_domain, seen):
    while True:
        try:
            url: str = queue.get()
        except Queue.Empty:
            logger.warn(f"que is empty")
            break

        if url is None:
            break

        # dont scrape pictures
        if url.endswith(".jpeg") or url.endswith(".jpg") or "email-protection" in url or 'Login?ReturnUrl' in url:
            logger.info(f'skipping {url}')
            queue.task_done()
            continue

        # shorten name
        url_no_https = url[8:]
        max_url_char = 200
        shortened_url = url_no_https[:max_url_char]

        filename = (
            "text/" + local_domain + "/" + shortened_url.replace("/", "_") + ".txt"
        )


        try:
            # Save text from the url to a <url>.txt file
            if not os.path.exists(filename):
                with open(filename, "w", encoding="utf-8") as f:


                    # Get the text from the URL using BeautifulSoup
                    logger.info(f"getting the url {url}")
                    response = requests.get(url, verify=False)
                    content_type = response.headers.get('Content-Type')

                    # write pdf instead of saving as text
                    if content_type and 'pdf' in content_type.lower():
                        text = read_pdf_from_url(url)
                        logger.info(f"writing text from pdf {url}")
                        f.write(text)
                        queue.task_done()
                        continue

                    soup = BeautifulSoup(response.text, "html.parser")

                    # Get the text but remove the tags

                    text = soup.get_text()


                    # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                    if "You need to enable JavaScript to run this app." in text:
                        print(
                            "Unable to parse page "
                            + url
                            + " due to JavaScript being required"
                        )
                        queue.task_done()
                        logger.warn(f"skipping url due to js {url}")
                        continue

                    # Otherwise, write the text to the file in the text directory
                    logger.info(f"writing text {url}")
                    f.write(text)
            else:
                logger.info(f'skipping file since it exists already {url}')

        except Exception as e:
            logger.error(f"Error writing to file {filename}: {e}")

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                logger.info(f'adding link {link}')
                queue.put(link)
                seen.add(link)

        queue.task_done()


def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = Queue()
    queue.put(url)

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
        os.mkdir("text/")

    if not os.path.exists("text/" + local_domain + "/"):
        os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
        os.mkdir("processed")


    # Create worker threads
    num_worker_threads = 12
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(queue, local_domain, seen))
        t.start()
        threads.append(t)

    # Wait for all tasks to be completed
    queue.join()

    # Stop workers
    for i in range(num_worker_threads):
        queue.put(None)
    for t in threads:
        t.join()


crawl(full_url)

In [ ]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [ ]:
import pandas as pd


# Create a list to store the text files
texts=[]


# Get all the text files in the text directory
for file in os.listdir("text/" + domain + "/"):
    print

    # Open the file and read the text
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()

        # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
        texts.append((file[11:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns = ['fname', 'text'])

# Set the text column to be the raw text with the newlines removed
df['text'] = df.fname + ". " + remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()

In [ ]:
import tiktoken

# Load the cl100k_base tokenizer which is designed to work with the ada-002 model
tokenizer = tiktoken.get_encoding("cl100k_base")

df = pd.read_csv('processed/scraped.csv', index_col=0)
df.columns = ['title', 'text']

# Tokenize the text and save the number of tokens to a new column
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

# Visualize the distribution of the number of tokens per row using a histogram
df.n_tokens.hist()

In [ ]:
max_tokens = 500

# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, max_tokens = max_tokens):

    # Split the text into sentences
    sentences = text.split('. ')

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater 
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of 
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks
    

shortened = []

# Loop through the dataframe
for row in df.iterrows():

    # If the text is None, go to the next row
    if row[1]['text'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the text into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])
    
    # Otherwise, add the text to the list of shortened texts
    else:
        shortened.append( row[1]['text'] )

In [ ]:
df = pd.DataFrame(shortened, columns = ['text'])
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
df.n_tokens.hist()

In [ ]:
import openai

df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])
df.to_csv('processed/embeddings.csv')
df.head()

In [ ]:
import pandas as pd
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

df=pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

df.head()

In [ ]:
def create_context(
    question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

def answer_question(
    df,
    model="text-davinci-003",
    question="Am I allowed to publish model outputs to Twitter, without a human review?",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.Completion.create(
            prompt=f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

In [ ]:
answer_question(df, question="What day is it?", debug=False)

In [ ]:
answer_question(df, question="What is our newest embeddings model?")